In [ ]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

## add electricity

In [ ]:
# network_path = os.getcwd() + '/networks/elec.nc'
network_path = os.getcwd() + '/networks/base.nc'
n = pypsa.Network(network_path)
n

In [ ]:
from shapely import wkt

lines = n.lines
lines['geometry'] = lines['geometry'].apply(wkt.loads)
lines = gpd.GeoDataFrame(lines, crs='epsg:4326')

buses = n.buses
buses["geometry"] = gpd.points_from_xy(buses.lon, buses.lat)
buses = gpd.GeoDataFrame(buses, crs='epsg:4326')

In [ ]:
# lines = lines[lines["country"]=="nigeria"]
# buses = buses[buses["country"]=="nigeria"]

buses.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## Simplify network

In [ ]:
network_path_s = os.getcwd() + '/networks/elec_s.nc'
n_s = pypsa.Network(network_path_s)
n_s

In [ ]:
from shapely import wkt

lines_s = n_s.lines
lines_s['geometry'] = lines_s['geometry'].apply(wkt.loads)
lines_s = gpd.GeoDataFrame(lines_s, crs='epsg:4326')

buses_s = n_s.buses
buses_s["geometry"] = gpd.points_from_xy(buses_s.lon, buses_s.lat)
buses_s = gpd.GeoDataFrame(buses_s, crs='epsg:4326')

buses_s.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_s.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## cluster_network

In [ ]:
network_path_c = os.getcwd() + '/networks/elec_s_50.nc'
n_c = pypsa.Network(network_path_c)
n_c

In [ ]:
from shapely import wkt
from shapely.geometry import LineString

buses_c = n_c.buses
buses_c["geometry"] = gpd.points_from_xy(buses_c.x, buses_c.y)
buses_c = gpd.GeoDataFrame(buses_c, crs='epsg:4326')

lines_c = n_c.lines
lines_c['geometry'] = lines_c.apply(lambda x: LineString([buses_c.loc[x["bus0"], "geometry"], buses_c.loc[x["bus1"], "geometry"]]), axis=1)
lines_c = gpd.GeoDataFrame(lines_c, crs='epsg:4326')

buses_c.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_c.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)